llm

#LLM

In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install -U bitsandbytes
!pip install PyPDF2
from unsloth import FastLanguageModel
import torch
from google.colab import files
import PyPDF2


Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-6j6nybnr
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-6j6nybnr
  Resolved https://github.com/unslothai/unsloth.git to commit f6174818d6f3461ff5b8457ddd514f350866d32e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.5-py3-none-any.whl size=174716 sha256=076aa17ca1572e68bff2dead5816c9a331e2419e7666ea264f62d1bfb857ff3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0vp1233k/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make train

In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'financial_ratios_with_altman.xlsx'  # Update with your file's path in Colab
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Define the required columns
required_columns = [
    "Symbol", "Year", "Current Ratio", "Quick Ratio", "Immediate Liquidity",
    "Cash Flow to Sales", "Net Profit Margin", "Operating Margin", "EBITDA Margin",
    "ROE", "ROA", "Leverage Ratio", "Equity to Total Assets", "Receivables Ratio",
    "Days Sales Outstanding", "Inventory Ratio", "Inventory Turnover",
    "Days Inventory", "Days Payable", "Altman Z-Score"
]

# Filter the DataFrame to include only the required columns
filtered_df = df[required_columns]

# Specify the company symbol and year
symbol = "EVGN.TA"
year = 2022

# Filter the data for the specified symbol and year
filtered_company_data = filtered_df[(filtered_df["Symbol"] == symbol) & (filtered_df["Year"] == year)]

# Format the data for presentation
formatted_data = filtered_company_data.transpose()
formatted_data.columns = ["Value"]
formatted_data.drop(["Symbol", "Year"], inplace=True)

# Display the formatted data
formatted_data.reset_index(inplace=True)
formatted_data.columns = ["Metric", "Value"]
print(formatted_data)


                    Metric       Value
0            Current Ratio    6.711289
1              Quick Ratio    6.610667
2      Immediate Liquidity    6.285333
3       Cash Flow to Sales  -14.136119
4        Net Profit Margin  -15.903284
5         Operating Margin  -17.665075
6            EBITDA Margin  -14.586866
7                      ROE   -0.953741
8                      ROA   -0.474611
9           Leverage Ratio         NaN
10  Equity to Total Assets     0.49763
11       Receivables Ratio    0.479403
12  Days Sales Outstanding   75.832836
13         Inventory Ratio    0.622662
14      Inventory Turnover    1.606007
15          Days Inventory  227.271727
16            Days Payable    415.9956
17          Altman Z-Score   -7.117745


In [ ]:
brief= "Generate a briefing that introduces the company. The briefing may focus on key aspects such as the company�s operations, products (if mentioned), its history and the company today."
question = """Analyze the provided financial statement and summarize key financial insights, focusing on revenue trends, profitability, cash flow, and significant investments. Identify any potential risks or challenges the company might face based on the data. Provide actionable insights that would be valuable to investors or financial analysts."""

In [ ]:
# Create the financial metrics prompt using the formatted_data DataFrame
financial_metrics_prompt = "**Financial Metrics:**\n\n"
for _, row in formatted_data.iterrows():
    financial_metrics_prompt += f"- **{row['Metric']}**: {row['Value']}\n"

In [ ]:
financial_metrics_prompt

'**Financial Metrics:**\n\n- **Current Ratio**: 6.711288888888889\n- **Quick Ratio**: 6.610666666666667\n- **Immediate Liquidity**: 6.285333333333333\n- **Cash Flow to Sales**: -14.13611940298508\n- **Net Profit Margin**: -15.90328358208955\n- **Operating Margin**: -17.66507462686567\n- **EBITDA Margin**: -14.58686567164179\n- **ROE**: -0.9537414965986395\n- **ROA**: -0.4746106973595125\n- **Leverage Ratio**: nan\n- **Equity to Total Assets**: 0.4976303317535545\n- **Receivables Ratio**: 0.4794029850746269\n- **Days Sales Outstanding**: 75.83283582089553\n- **Inventory Ratio**: 0.6226622662266227\n- **Inventory Turnover**: 1.606007067137809\n- **Days Inventory**: 227.2717271727173\n- **Days Payable**: 415.995599559956\n- **Altman Z-Score**: -7.117744816995763\n'

swot llm

In [ ]:
prompt_swot = """You are an advanced LLM tasked with creating a SWOT analysis for a company based on the provided context. Use the following information to develop a detailed SWOT model, identifying the company's Strengths, Weaknesses, Opportunities, and Threats. Ensure that each section is clearly articulated and reflects insights derived from the context."""
prompt_swot = """You are a highly capable LLM tasked with creating a detailed SWOT analysis for Evogene based on the provided context. Your analysis should identify and categorize key aspects into Strengths, Weaknesses, Opportunities, and Threats. Each section must include at least three examples where applicable. Present your findings in the specified format below.


Task:

Analyze the provided context.
Develop a comprehensive SWOT analysis.
Format your analysis as follows, ensuring each section contains at least three examples:
Strengths:

[Example 1]
[Example 2]
[Example 3]
Weaknesses:

[Example 1]
[Example 2]
[Example 3]
Opportunities:

[Example 1]
[Example 2]
[Example 3]
Threats:

[Example 1]
[Example 2]
[Example 3]
Ensure that each point is clearly articulated, drawing directly from the context to reflect a thorough understanding of the company's current situation and market environment."""

In [ ]:

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    text = ""
    try:
        with open(file_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text

# Upload the PDF file
print("Please upload a PDF file:")
uploaded = files.upload()

# Get the uploaded file name
pdf_file_name = list(uploaded.keys())[0]

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_name)

if not extracted_text.strip():
    print("No text found in the uploaded PDF.")
else:
    print("PDF text successfully extracted.")

    # Load the pre-trained model and tokenizer
    model_path = "meta-llama/Llama-3.2-1B-Instruct"  # Path to the locally saved model
    print(f"Loading the model from {model_path}...")
    model, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=False)

    # Prepare the model for inference
    model = FastLanguageModel.for_inference(model)

    # Move the model to the GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("Embedding the PDF text as the model's knowledge base...")

    # Tokenize the extracted text
    text_inputs = tokenizer(extracted_text, return_tensors="pt", truncation=True, max_length=2048)
    text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

    print("Model is ready. You can now ask questions about the PDF content.")
    i=0
    while i==0:
        i=1
        question = question
        if question.lower() == 'exit':
            print("Exiting...")
            break

        # Directly use the question without embedding the full context
        input_text = (
            f"Context: {extracted_text}\n\n"  # Truncate context to fit the token limit
            f"{financial_metrics_prompt}\n"
            f"Question: {prompt_swot}\n"
            "Answer the question concisely based on the context provided.:"
        )
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}


        # Generate the model's response
        with torch.no_grad():
            outputs = model.generate(**inputs)

        # Decode and print the model's response
        response_swot = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Response from the model:")
        response_swot = response_swot.split("based on the context provided.:")[1]
        print(response_swot)

Please upload a PDF file:


Saving a.pdf to a (8).pdf
PDF text successfully extracted.
Loading the model from meta-llama/Llama-3.2-1B-Instruct...
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Embedding the PDF text as the model's knowledge base...
Model is ready. You can now ask questions about the PDF content.
Response from the model:



**Detailed SWOT Analysis for Evogene:**

**Strengths:**

1. **Computational Predictive Biology (CPB) Platform**: Evogene's CPB platform is a key differentiator in the life sciences industry, offering a unique approach to discovering and developing life-science products.
2. **Inn

simple llm

In [ ]:

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    text = ""
    try:
        with open(file_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text

# Upload the PDF file
print("Please upload a PDF file:")
uploaded = files.upload()

# Get the uploaded file name
pdf_file_name = list(uploaded.keys())[0]

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_name)

if not extracted_text.strip():
    print("No text found in the uploaded PDF.")
else:
    print("PDF text successfully extracted.")

    # Load the pre-trained model and tokenizer
    model_path = "meta-llama/Llama-3.2-1B-Instruct"  # Path to the locally saved model
    print(f"Loading the model from {model_path}...")
    model, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=False)

    # Prepare the model for inference
    model = FastLanguageModel.for_inference(model)

    # Move the model to the GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("Embedding the PDF text as the model's knowledge base...")

    # Tokenize the extracted text
    text_inputs = tokenizer(extracted_text, return_tensors="pt", truncation=True, max_length=2048)
    text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

    print("Model is ready. You can now ask questions about the PDF content.")
    i=0
    while i==0:
        i=1
        question = question
        if question.lower() == 'exit':
            print("Exiting...")
            break

        # Directly use the question without embedding the full context
        input_text = (
            f"Context: {extracted_text}\n\n"  # Truncate context to fit the token limit
            f"{financial_metrics_prompt}\n"
            f"Question: {brief}\n"
            f"{question}\n"
            "Answer the question concisely based on the context provided.:"
        )
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}


        # Generate the model's response
        with torch.no_grad():
            outputs = model.generate(**inputs)

        # Decode and print the model's response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Response from the model:")
        response = response.split("based on the context provided.:")[1]
        print(response)

Please upload a PDF file:


IndexError: list index out of range

In [ ]:
!pip install python-pptx


In [ ]:
presentation_prompt = """
always give me the following stracture for the contex:
**Title Slide**: Include the company name and stock symbol.
**Company Overview**: Summarize the company's mission, products, and focus areas.
**Key Financial Insights**: Highlight financial metrics like liquidity, profitability, and growth trends.
**Risks and Challenges**: Outline key challenges such as competition or regulatory changes.
**Actionable Insights**: Provide recommendations for overcoming challenges and leveraging growth opportunities.
"""


In [ ]:
    # Load the pre-trained model and tokenizer
    model_path = "meta-llama/Llama-3.2-1B-Instruct"  # Path to the locally saved model
    print(f"Loading the model from {model_path}...")
    model, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=False)

    # Prepare the model for inference
    model = FastLanguageModel.for_inference(model)

    # Move the model to the GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("Embedding the PDF text as the model's knowledge base...")

    # Tokenize the extracted text
    text_inputs = tokenizer(extracted_text, return_tensors="pt", truncation=True, max_length=2048)
    text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

    print("Model is ready. You can now ask questions about the PDF content.")
    i=0
    while i==0:
        i=1
        question = question
        if question.lower() == 'exit':
            print("Exiting...")
            break

        # Directly use the question without embedding the full context
        input_text = (
            f"Context: {response}\n\n"  # Truncate context to fit the token limit
            f"{presentation_prompt}\n"
            "Answer the question concisely based on the context provided.:"
        )
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}


        # Generate the model's response
        with torch.no_grad():
            outputs = model.generate(**inputs)

        # Decode and print the model's response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Response from the model:")
        response = response.split("based on the context provided.:")[1]
        print(response)

Loading the model from meta-llama/Llama-3.2-1B-Instruct...
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Embedding the PDF text as the model's knowledge base...
Model is ready. You can now ask questions about the PDF content.
Response from the model:



**Title Slide**: Evogene Ltd.
**Company Overview**: Evogene Ltd. is a biotechnology company that focuses on developing computational predictive biology (CPB) solutions for various industries, including life sciences, agriculture, and industrial applications. The company was founded in 2000 and has since grown into a leading player in th

finance only

In [ ]:
finance_prompt = """You are a highly capable LLM tasked with analyzing a set of financial metrics provided for a company. Your analysis should include an evaluation of each metric individually and a comprehensive assessment of all the metrics together. The goal is to provide insights into the company's financial health, operational efficiency, liquidity, profitability, and overall risk.

*Task:*
1. *Individual Metric Analysis:*
   - Analyze each financial metric separately (e.g., Current Ratio, Quick Ratio, etc.).
   - Explain what each metric indicates about the company's financial condition.

2. *Holistic Assessment:*
   - Evaluate the company's overall financial health by considering all metrics together.
   - Identify patterns, potential risks, strengths, and weaknesses."""

In [ ]:
    # Load the pre-trained model and tokenizer
    model_path = "meta-llama/Llama-3.2-1B-Instruct"  # Path to the locally saved model
    print(f"Loading the model from {model_path}...")
    model, tokenizer = FastLanguageModel.from_pretrained(model_path, load_in_4bit=False)

    # Prepare the model for inference
    model = FastLanguageModel.for_inference(model)

    # Move the model to the GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("Embedding the PDF text as the model's knowledge base...")

    # Tokenize the extracted text
    text_inputs = tokenizer(extracted_text, return_tensors="pt", truncation=True, max_length=2048)
    text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

    print("Model is ready. You can now ask questions about the PDF content.")
    i=0
    while i==0:
        i=1
        question = question
        if question.lower() == 'exit':
            print("Exiting...")
            break

        # Directly use the question without embedding the full context
        input_text = (
            f"{financial_metrics_prompt}\n"
            f"{finance_prompt}\n"
            "Answer the question concisely based on the context provided.:"
        )
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}


        # Generate the model's response
        with torch.no_grad():
            outputs = model.generate(**inputs)

        # Decode and print the model's response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Response from the model:")
        response = response.split("based on the context provided.:")[1]
        print(response)

Loading the model from meta-llama/Llama-3.2-1B-Instruct...
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Embedding the PDF text as the model's knowledge base...
Model is ready. You can now ask questions about the PDF content.
Response from the model:




## Step 1: Analyze the Current Ratio
The current ratio is calculated by dividing current assets by current liabilities. A current ratio of 6.711288888888889 indicates that the company has sufficient current assets to cover its current liabilities, suggesting a relatively healthy financial position.

## Step 2: Analyze the Quick Ratio
T

In [ ]:
import re

# Function to preprocess text and extract sections
def preprocess_text(text):
    """Remove newlines and extra spaces from the input text."""
    return re.sub(r'\s+', ' ', text).strip()

def extract_section(text, start_marker, end_marker=None):
    """Extract a section of text between start_marker and end_marker."""
    try:
        start_index = text.find(start_marker) + len(start_marker)
        if end_marker:
            end_index = text.find(end_marker, start_index)
            return text[start_index:end_index].strip()
        return text[start_index:].strip()
    except Exception as e:
        print(f"Error extracting section: {e}")
        return ""

In [ ]:
resp = preprocess_text(response)

In [ ]:
resp

"Please provide the following structure for the context: ## Step 1: Company Overview Evogene Ltd. is a leading computational biology company targeting life-science based industries. It offers a range of computational predictive biology products aimed at developing novel therapeutics and diagnostics. ## Step 2: Key Financial Insights - **Current Ratio**: 6.711288888888889 - **Quick Ratio**: 6.610666666666667 - **Immediate Liquidity**: 6.285333333333333 - **Cash Flow to Sales**: -14.13611940298508 - **Net Profit Margin**: -15.90328358208955 - **Operating Margin**: -17.66507462686567 - **EBITDA Margin**: -14.58686567164179 - **ROE**: -0.9537414965986395 - **ROA**: -0.4746106973595125 - **Leverage Ratio**: nan - **Equity to Total Assets**: 0.4976303317535545 - **Receivables Ratio**: 0.4794029850746269 - **Days Sales Outstanding**: 75.83283582089553 - **Inventory Ratio**: 0.6226622662266227 - **Inventory Turnover**: 1.606007067137809 - **Days Inventory**: 227.2717271727173 - **Days Payable*

In [ ]:
resp_swot = preprocess_text(response_swot)

In [ ]:
resp_swot

"**Detailed SWOT Analysis for Evogene:** **Strengths:** 1. **Computational Predictive Biology (CPB) Platform**: Evogene's CPB platform is a key differentiator in the life sciences industry, offering a unique approach to discovering and developing life-science products. 2. **Innovative Research and Development**: Evogene has a strong research and development pipeline, with a focus on innovative technologies and approaches that can drive breakthroughs in the field. 3. **Strong Financial Performance**: Evogene has consistently demonstrated strong financial performance, with a focus on generating revenue and profits through its life-science products. **Weaknesses:** 1. **High R&D Costs**: The high costs associated with R&D can be a challenge for Evogene, particularly in a market where competition is fierce. 2. **Limited Commercialization Success**: While Evogene has had success with its CPB platform, there is still a need for commercialization success to drive revenue growth. 3. **Dependen

#Presentation-Creation

In [ ]:
# Generate a PowerPoint presentation WORKS WITH SWOT
from pptx.util import Pt

def create_presentation(company_name, stock_symbol, company_description, key_insights, risks, actions, financial_metrics, swot_analysis):
    prs = Presentation()
    stock = yf.Ticker(stock_symbol)

    # Title Slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = f"{company_name} Financial Presentation"
    subtitle.text = f"Analysis for {stock_symbol}"

    # Company Overview Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Company Overview"
    content = slide.placeholders[1]
    content.text = company_description

    # Key Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Key Financial Insights"
    content = slide.placeholders[1]
    content.text = key_insights

    # Risks and Challenges Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Risks and Challenges"
    content = slide.placeholders[1]
    content.text = risks

    # Actionable Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Actionable Insights"
    content = slide.placeholders[1]
    content.text = actions


    # Create a text box for better control over text formatting
    left = Inches(1)
    top = Inches(1.5)
    width = Inches(8)
    height = Inches(5)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True

    # Populate the metrics with smaller font size
    for _, row in financial_metrics.iterrows():
        paragraph = text_frame.add_paragraph()
        paragraph.text = f"{row['Metric']}: {row['Value']}"
        paragraph.font.size = Pt(12)

    # SWOT Analysis Slides
    strengths = extract_section(swot_analysis, "**Strengths:**", "**Weaknesses:**")
    weaknesses = extract_section(swot_analysis, "**Weaknesses:**", "**Opportunities:**")
    opportunities = extract_section(swot_analysis, "**Opportunities:**", "**Threats:**")
    threats = extract_section(swot_analysis, "**Threats:**", "")

    # Add Strengths Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "SWOT Analysis: Strengths"
    left = Inches(1)
    top = Inches(1.5)
    width = Inches(8)
    height = Inches(5)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True
    for line in strengths.split("\n"):
        paragraph = text_frame.add_paragraph()
        paragraph.text = line.strip()
        paragraph.font.size = Pt(12)  # Smaller font size

    # Add Weaknesses Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "SWOT Analysis: Weaknesses"
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True
    for line in weaknesses.split("\n"):
        paragraph = text_frame.add_paragraph()
        paragraph.text = line.strip()
        paragraph.font.size = Pt(12)  # Smaller font size

    # Add Opportunities Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "SWOT Analysis: Opportunities"
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True
    for line in opportunities.split("\n"):
        paragraph = text_frame.add_paragraph()
        paragraph.text = line.strip()
        paragraph.font.size = Pt(12)  # Smaller font size

    # Add Threats Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "SWOT Analysis: Threats"
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True
    for line in threats.split("\n"):
        paragraph = text_frame.add_paragraph()
        paragraph.text = line.strip()
        paragraph.font.size = Pt(12)  # Smaller font size

    # Line Chart Slide
    x = [1, 2, 3, 4]
    y = [([stock.income_stmt.loc['Total Revenue'].iloc[i] for i in range(4)], 'Total Revenue'),
         ([stock.balance_sheet.loc['Cash And Cash Equivalents'].iloc[i] for i in range(4)], 'Cash And Cash Equivalents')]

    generate_line_chart(x, y)
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Line Chart"
    chart_path = "line_chart.png"
    slide.shapes.add_picture(chart_path, Inches(1), Inches(1), width=Inches(5))

    # Save the presentation
    prs.save(f'{stock_symbol}_financial_presentation.pptx')


# Extract the SWOT analysis sections
swot_analysis = resp_swot
strengths = extract_section(swot_analysis, "**Strengths:**", "**Weaknesses:**")
weaknesses = extract_section(swot_analysis, "**Weaknesses:**", "**Opportunities:**")
opportunities = extract_section(swot_analysis, "**Opportunities:**", "**Threats:**")
threats = extract_section(swot_analysis, "**Threats:**", "")


financial_metrics_df = pd.DataFrame()


# Generate the presentation with the SWOT analysis included
create_presentation(
    company_name,
    "EVGN",
    company_description,
    key_insights,
    risks,
    actions,
    financial_metrics_df,
    swot_analysis
)


In [ ]:

from pptx.util import Pt

def create_presentation(company_name, stock_symbol, company_description, key_insights, risks, actions, financial_metrics, swot_analysis):
    prs = Presentation()
    stock = yf.Ticker(stock_symbol)

    # Title Slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = f"{company_name} Financial Presentation"
    subtitle.text = f"Analysis for {stock_symbol}"

    # Company Overview Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Company Overview"
    content = slide.placeholders[1]
    content.text = company_description

    # Key Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Key Financial Insights"
    content = slide.placeholders[1]
    content.text = key_insights

    # Risks and Challenges Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Risks and Challenges"
    content = slide.placeholders[1]
    content.text = risks

    # Actionable Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Actionable Insights"
    content = slide.placeholders[1]
    content.text = actions


    # Create a text box for better control over text formatting
    left = Inches(1)
    top = Inches(1.5)
    width = Inches(8)
    height = Inches(5)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True

    # Populate the metrics with smaller font size
    for _, row in financial_metrics.iterrows():
        paragraph = text_frame.add_paragraph()
        paragraph.text = f"{row['Metric']}: {row['Value']}"
        paragraph.font.size = Pt(12)

    # SWOT Analysis Slides
    strengths = extract_section(swot_analysis, "**Strengths:**", "**Weaknesses:**")
    weaknesses = extract_section(swot_analysis, "**Weaknesses:**", "**Opportunities:**")
    opportunities = extract_section(swot_analysis, "**Opportunities:**", "**Threats:**")
    threats = extract_section(swot_analysis, "**Threats:**", "")

    # Add SWOT Analysis Slides with proper formatting
    def add_swot_slide(prs, title_text, swot_text):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        title = slide.shapes.title
        title.text = title_text

        # Create a text box for better control over text formatting
        left = Inches(1)
        top = Inches(1.5)
        width = Inches(8)
        height = Inches(5)
        text_box = slide.shapes.add_textbox(left, top, width, height)
        text_frame = text_box.text_frame
        text_frame.word_wrap = True

        # Split the text into lines based on numbered items
        for line in swot_text.split("**"):
            if line.strip():
                paragraph = text_frame.add_paragraph()
                paragraph.text = line.strip()
                paragraph.font.size = Pt(14)  # Adjust font size
                paragraph.space_after = Pt(10)  # Add space after each line

    # Add Strengths Slide
    add_swot_slide(prs, "SWOT Analysis: Strengths", strengths)

    # Add Weaknesses Slide
    add_swot_slide(prs, "SWOT Analysis: Weaknesses", weaknesses)

    # Add Opportunities Slide
    add_swot_slide(prs, "SWOT Analysis: Opportunities", opportunities)

    # Add Threats Slide
    add_swot_slide(prs, "SWOT Analysis: Threats", threats)

    # Line Chart Slide
    x = [1, 2, 3, 4]
    y = [([stock.income_stmt.loc['Total Revenue'].iloc[i] for i in range(4)], 'Total Revenue'),
         ([stock.balance_sheet.loc['Cash And Cash Equivalents'].iloc[i] for i in range(4)], 'Cash And Cash Equivalents')]

    generate_line_chart(x, y)
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Line Chart"
    chart_path = "line_chart.png"
    slide.shapes.add_picture(chart_path, Inches(1), Inches(1), width=Inches(5))

    # Save the presentation
    prs.save(f'{stock_symbol}_financial_presentation.pptx')


# Extract the SWOT analysis sections
swot_analysis = resp_swot
strengths = extract_section(swot_analysis, "**Strengths:**", "**Weaknesses:**")
weaknesses = extract_section(swot_analysis, "**Weaknesses:**", "**Opportunities:**")
opportunities = extract_section(swot_analysis, "**Opportunities:**", "**Threats:**")
threats = extract_section(swot_analysis, "**Threats:**", "")


financial_metrics_df = pd.DataFrame()


# Generate the presentation with the SWOT analysis included
create_presentation(
    company_name,
    "EVGN",
    company_description,
    key_insights,
    risks,
    actions,
    financial_metrics_df,
    swot_analysis
)


In [ ]:
#works


import matplotlib.pyplot as plt
import pandas as pd
from pptx import Presentation
from pptx.util import Inches

import yfinance as yf



def generate_line_chart(x,y):
    # Generate a simple line chart

    for line,label in y:
     plt.plot(x, line, label=label)

    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.title('Simple Line Chart')
    plt.legend()

    # Save the chart as an image
    plt.savefig('line_chart.png')
    plt.close()

# Extract sections from response
def extract_section(text, start_marker, end_marker=None):
    start_index = text.find(start_marker) + len(start_marker)
    if end_marker:
        end_index = text.find(end_marker, start_index)
        return text[start_index:end_index].strip()
    return text[start_index:].strip()

# Generate a PowerPoint presentation
def create_presentation(company_name, stock_symbol, company_description, key_insights, risks, actions, financial_metrics):
    prs = Presentation()
    stock = yf.Ticker(stock_symbol)

    # Title Slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = f"{company_name} Financial Presentation"
    subtitle.text = f"Analysis for {stock_symbol}"

    # Company Overview Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Company Overview"
    content = slide.placeholders[1]
    content.text = company_description

    # Key Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Key Financial Insights"
    content = slide.placeholders[1]
    content.text = key_insights

    # Risks and Challenges Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Risks and Challenges"
    content = slide.placeholders[1]
    content.text = risks

    # Actionable Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Actionable Insights"
    content = slide.placeholders[1]
    content.text = actions

    # Financial Metrics Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Financial Metrics"

    # Create a text box for better control over text formatting
    left = Inches(1)
    top = Inches(1.5)
    width = Inches(8)
    height = Inches(5)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame
    text_frame.word_wrap = True

    # Populate the metrics with smaller font size
    for _, row in financial_metrics.iterrows():
        paragraph = text_frame.add_paragraph()
        paragraph.text = f"{row['Metric']}: {row['Value']}"
        paragraph.font.size = Pt(12)  # Adjust the font size here (e.g., 12pt)


    # Line Chart Slide
    x = [1, 2, 3, 4]
    y= [([stock.income_stmt.loc['Total Revenue'].iloc[i] for i in range(4)],'Total Revenue'),
        ([stock.balance_sheet.loc['Cash And Cash Equivalents'].iloc[i] for i in range(4)],'Cash And Cash Equivalents')]

    generate_line_chart(x,y)
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Line Chart"
    chart_path = "line_chart.png"
    slide.shapes.add_picture(chart_path, Inches(1), Inches(1), width=Inches(5))

    # Save the presentation
    prs.save(f'{stock_symbol}_financial_presentation.pptx')

# Extract arguments dynamically from the response
company_name = extract_section(resp, "**Title Slide**: ", " **Company Overview**")
company_description = extract_section(resp, "**Company Overview**: ", " **Key Financial Insights**")
key_insights = extract_section(resp, "**Key Financial Insights**: ", " **Risks and Challenges**")
risks = extract_section(resp, "**Risks and Challenges**: ", " **Actionable Insights**")
actions = extract_section(resp, "**Actionable Insights**: ", " **Financial Metrics**")

# Extract Financial Metrics
metrics_start = resp.find("**Financial Metrics**: ") + len("**Financial Metrics**: ")
metrics_text = resp[metrics_start:].split(" **Data Visualization**")[0].strip()

# Convert financial metrics to DataFrame
financial_metrics = []
for line in metrics_text.split(" - "):
    if ": " in line:
        metric, value = line.split(": ", 1)
        financial_metrics.append((metric.strip(), float(value) if value != "nan" else None))
financial_metrics_df = pd.DataFrame(financial_metrics, columns=["Metric", "Value"])

# Generate the chart and presentation

create_presentation(company_name, "EVGN", company_description, key_insights, risks, actions, financial_metrics_df)

ValueError: could not convert string to float: "Evogene Ltd. is a biotechnology company that focuses on developing computational predictive biology (CPB) solutions for various industries, including life sciences, agriculture, and industrial applications. The company was founded in 2000 and has since grown into a leading player in the CPB space. **Key Financial Insights**: Evogene's financial statements for the quarter ended September 30, 2020, show a significant increase in revenue compared to the same period in the previous year. The company reported a net profit of $17.4 million, which is a 20% increase from the same period in the previous year. The company's operating margin was 17.7%, which is a 50 basis point increase from the same period in the previous year. **Risks and Challenges**: Evogene faces several risks and challenges, including increased competition in the life sciences industry, regulatory hurdles in some of its markets, and the impact of the COVID-19 pandemic on its b

In [ ]:
key_insights

'- Revenue growth: 3% YoY - Net income: 4% YoY - Operating margin: 12% YoY - Return on equity (ROE): 15% - Cash flow from operations: 6% of revenue'

In [ ]:
# Example Usage with a dynamic response
response = """
**Title Slide**: Evogene Ltd.

**Company Overview**: Evogene Ltd. is a leading computational biology company focused on developing life science products for various markets, including human health, agriculture, and industrial applications. The company was founded in 2002 and is headquartered in Rehovot, Israel.

**Key Financial Insights**: Evogene's revenue has been steadily increasing over the years, driven by the growth of the life science industry. The company's operating margin has improved, driven by cost savings and efficiency gains. However, the company's cash flow has been somewhat constrained, with a net cash outflow of $50 million in the past year.

**Risks and Challenges**: The company faces competition from established players in the life science industry, as well as regulatory changes that may impact its products and business model.

**Actionable Insights**: To overcome these challenges, Evogene should focus on expanding its product portfolio, acquiring new technologies, and investing in marketing and sales efforts. The company should also monitor its cash flow closely and adjust its spending and investments to ensure that it is operating within its means.

**Financial Metrics**:

- Current Ratio: 6.711288888888889
- Quick Ratio: 6.610666666666667
- Immediate Liquidity: 6.285333333333333
- Cash Flow to Sales: -14.13611940298508
- Net Profit Margin: -15.90328358208955
- Operating Margin: -17.66507462686567
- EBITDA Margin: -14.58686567164179
- ROE: -0.9537414965986395
- ROA: -0.4746106973595125
- Leverage Ratio: nan
- Equity to Total Assets: 0.4976303317535545
- Receivables Ratio: 0.4794029850746269
- Days Sales Outstanding: 75.83283582089553
- Inventory Ratio: 0.6226622662266227
- Inventory Turnover: 1.606007067137809
- Days Inventory: 227.2717271727173
- Days Payable: 415.995599559956
- Altman Z-Score: -7.117744816995763
"""


In [ ]:
import matplotlib.pyplot as plt
import yfinance as yf
import requests
import pandas as pd
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

# Generate a simple line chart
def generate_line_chart():
    x = [1, 2, 3, 4]
    y1 = [10, 15, 13, 17]
    y2 = [5, 7, 8, 9]

    plt.plot(x, y1, label='Line 1')
    plt.plot(x, y2, label='Line 2')

    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.title('Simple Line Chart')
    plt.legend()

    plt.savefig('line_chart.png')
    plt.close()

# Generate a PowerPoint presentation
def create_presentation(company_name, stock_symbol, company_description, key_insights, risks, actions, financial_metrics):
    prs = Presentation()

    # Title Slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = f"{company_name} Financial Presentation"
    subtitle.text = f"Analysis for {stock_symbol}"

    # Company Overview Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Company Overview"
    content = slide.placeholders[1]
    content.text = company_description

    # Key Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Key Financial Insights"
    content = slide.placeholders[1]
    content.text = key_insights

    # Risks and Challenges Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Potential Risks and Challenges"
    content = slide.placeholders[1]
    content.text = risks

    # Actionable Insights Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Actionable Insights"
    content = slide.placeholders[1]
    content.text = actions

    # Financial Metrics Slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title = slide.shapes.title
    title.text = "Financial Metrics"
    content = slide.placeholders[1]
    metrics_text = ""
    for _, row in financial_metrics.iterrows():
        metrics_text += f"{row['Metric']}: {row['Value']}\n"
    content.text = metrics_text

    # Line Chart Slide
    slide = prs.slides.add_slide(prs.slide_layouts[5])
    title = slide.shapes.title
    title.text = "Revenue Trends (Sample Chart)"
    slide.shapes.add_picture('line_chart.png', Inches(1), Inches(1), width=Inches(6))

    # Save the presentation
    prs.save(f'{stock_symbol}_financial_presentation.pptx')

# Example Usage
generate_line_chart()  # Generate the line chart image

# Input data
company_name = "Evogene"
stock_symbol = "EVGN"
company_description = """Evogene is a leading developer of computational predictive biology products, with a strong focus on cancer and cardiovascular disease."""
key_insights = """Evogene's revenue has been steadily increasing, profitability improving, and cash flow is strong with a current ratio of 6.71."""
risks = """Evogene faces competition, challenges in scaling its business, and maintaining its competitive advantage."""
actions = """Evogene should focus on profitability, investing in key technologies, and maintaining its competitive edge."""

# Mocking financial metrics from `formatted_data` DataFrame
formatted_data = pd.DataFrame({
    "Metric": ["Current Ratio", "Quick Ratio", "Immediate Liquidity", "Net Profit Margin"],
    "Value": [6.71, 6.61, 6.29, -15.90]
})

# Create the presentation
create_presentation(company_name, stock_symbol, company_description, key_insights, risks, actions, formatted_data)
